In [1]:
import matplotlib.pyplot as plt
import tensorflow as tf
import matplotlib.gridspec as gridspec
import cv2
import matplotlib.image as mpimg
import os
import pickle
import pandas as pd
from sklearn.utils import shuffle
import numpy as np
from sklearn.model_selection import train_test_split
%matplotlib inline

## NEW =============================

In [ ]:
# Load Annie data

with open('training_data_annieonly.p', 'rb') as handle:
    training_data_annie = pickle.load(handle)
    df_annie = pd.DataFrame.from_dict(training_data_annie)
    

In [ ]:
X_train_data, X_valid_data = train_test_split(df, test_size = 0.2)


In [ ]:
X_train_data_recovery, X_valid_data_recovery = train_test_split(df_annie, test_size = 0.2)
print('X_train_data_recovery shape: ', X_train_data_recovery.shape)
print('X_valid_data_recovery shape: ', X_valid_data_recovery.shape)

In [ ]:
X_train_data_recovery.plot.hist('steer', bins=100,range=(-1,1),facecolor="r", histtype = 'step')

# New ==========================

In [ ]:
df.head(10)
print(len(df))
print(len(X_train_data))
print(len(X_valid_data))

In [ ]:
X_train_data.head(10)

#### View histogram of steering angles

In [ ]:
X_train_data.plot.hist('steer', bins=100,range=(-1,1),facecolor="r", histtype = 'step')

In [ ]:
row = X_train_data.iloc[[2]]
print(row['steer'].values[0])

#### As you can see, most of our steering angles are around zero

## Random Testing

In [ ]:
rand_int = np.random.choice(len(X_train_data))

In [ ]:
print('rand_int: ', rand_int)
row = X_train_data.iloc[[rand_int]]
impath = row['image'].values[0]
steer = row['steer'].values[0]
print('impath ', impath)
print('steer: ', steer)

img, ang = preprocess_image_from_path(impath, steer)
plt.imshow(img)
print('steer: ', ang)

# img = mpimg.imread(impath)
# plt.imshow(img)

In [ ]:
print('rand_int: ', rand_int)
row = X_train_data.iloc[[rand_int]]
impath = row['image'].values[0]
steer = row['steer'].values[0]
print('impath ', impath)
print('steer: ', steer)

img, ang = preprocess_image_from_path(impath, steer)
plt.imshow(img)
print('steer: ', ang)

# img = mpimg.imread(impath)
# plt.imshow(img)

## Brightness Augmentation

## Horizontal and Vertical Shifts

## Flipping

In [ ]:
def flip_image(image):
    image_flipped = np.fliplr(image)
    return image_flipped

## Preprocess

In [ ]:
def preprocess_image(image):
    """
    Preprocess image, 
    input: image (original shape)
    output: image (shape is (220, 66, 3) )
    """    
    # crop shape
    image = image[image.shape[0] * 0.34:image.shape[0] * 0.875,:,:]
    # resize to (66, 220)
    img = cv2.resize(image, (220, 66), interpolation=cv2.INTER_AREA)
    return img


In [ ]:
def preprocess_image_valid_from_path(image_path, steering_angle):
    img = mpimg.imread(image_path)
    img = preprocess_image(img)
    return img, steering_angle

In [ ]:
def preprocess_image_from_path(image_path, steering_angle):
    img = mpimg.imread(image_path)
    
    # chance to flip
#     prob_flip = np.random.randint(2)
#     if prob_flip == 1:
#         img = flip_image(img)
#         steering_angle = -1 * steering_angle
        
    img = preprocess_image(img)
    return img, steering_angle

## Generator

Here I created two generators, one for training data and one for validation data. In the training generator we create batches of 32 for each training sample. Therefore if I have `samples_per_epoch = 10` that means for each of those samples I yield 32 images from the training generator. I did this because it allows me to have control over what the batches turn out to be. In this case [ADD HERE]```add here I will add a probability function to pass a certain image path to the processor```

In [ ]:

def generate_training_data_old(data, batch_size = 32):
    """
    We create a loop through out data and 
    send out an individual row in the dataframe to preprocess_image_from_path, 
    which is then sent to preprocess_image
    inputs: 
    data: pandas DataFrame
    batch_size: batch sizes, size to make each batch
    returns a yield (image_batch, label_batch)
    """
    image_batch = np.zeros((batch_size, 66, 220, 3)) # nvidia input params
    label_batch = np.zeros((batch_size))
    while True:
        for i in range(batch_size):
            idx = np.random.randint(len(data))
            row = data.iloc[[idx]].reset_index()
            x, y = preprocess_image_from_path(row['image'].values[0], row['steer'].values[0])
            
            image_batch[i] = x
            label_batch[i] = y
        yield shuffle(image_batch, label_batch)
    
def generate_validation_data(data):
    while True:
        for idx in range(len(data)):
            row = data.iloc[[idx]].reset_index()
            img, angle = preprocess_image_valid_from_path(row['image'].values[0], row['steer'].values[0])
            img = img.reshape(1, img.shape[0], img.shape[1], img.shape[2])
            angle = np.array([[angle]])
            yield img, angle
            

In [ ]:

def generate_training_data(data, batch_size = 32):
    """
    We create a loop through out data and 
    send out an individual row in the dataframe to preprocess_image_from_path, 
    which is then sent to preprocess_image
    inputs: 
    data: pandas DataFrame
    batch_size: batch sizes, size to make each batch
    returns a yield (image_batch, label_batch)
    """
    image_batch = np.zeros((batch_size*2, 66, 220, 3)) # nvidia input params
    label_batch = np.zeros((batch_size*2))
    while True:
        for i in range(batch_size):
            idx = np.random.randint(len(data))
            row = data.iloc[[idx]].reset_index()
            x, y = preprocess_image_from_path(row['image'].values[0], row['steer'].values[0])
            
            # flip an image and return it
            x2 = np.fliplr(x)
            y2 = -1 * y
            
            image_batch[i] = x
            label_batch[i] = y
            image_batch[i+1] = x2
            image_batch[i+2] = y2
        yield shuffle(image_batch, label_batch)
    

            

In [ ]:
def generate_validation_data(data):
    while True:
        for idx in range(len(data)):
            row = data.iloc[[idx]].reset_index()
            img, angle = preprocess_image_valid_from_path(row['image'].values[0], row['steer'].values[0])
            img = img.reshape(1, img.shape[0], img.shape[1], img.shape[2])
            angle = np.array([[angle]])
            yield img, angle

## Network

#### I chose to use Nvidia's network architecture. Input (220 x 66 sized image) output (1 steering angle)

I chose to use the Nvidia model architecture which can be found [here add link]
I used ELu's because they push mean unit activation functions closer to zero [https://arxiv.org/pdf/1511.07289v1.pdf]

In [ ]:
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.core import Activation, Dropout, Flatten, Dense, Lambda
from keras.layers import ELU
from keras.optimizers import Adam
tf.python.control_flow_ops = tf


N_img_height = 66
N_img_width = 220
N_img_channels = 3
def nvidia_model():
    inputShape = (N_img_height, N_img_width, N_img_channels)

    model = Sequential()
    # normalization
    model.add(Lambda(lambda x: x / 127.5 - 1, input_shape = (66, 220, 3)))
    # cropping 70 off top 25 off bottom
    # model.add(Cropping2D(cropping=((70,25), (0, 0)))) Probably going to do cropping in my process

    # subsample is strides
    model.add(Convolution2D(24, 5, 5, 
                            subsample=(2,2), 
                            border_mode = 'valid',
                            init = 'he_normal',
                            name = 'conv1'))
    
    model.add(ELU())    
    model.add(Convolution2D(36, 5, 5, 
                            subsample=(2,2), 
                            border_mode = 'valid',
                            init = 'he_normal',
                            name = 'conv2'))
    
    model.add(ELU())    
    model.add(Convolution2D(48, 5, 5, 
                            subsample=(2,2), 
                            border_mode = 'valid',
                            init = 'he_normal',
                            name = 'conv3'))
    model.add(ELU())
    model.add(Convolution2D(64, 3, 3, 
                            subsample = (1,1), 
                            border_mode = 'valid',
                            init = 'he_normal', #gaussian init
                            name = 'conv4'))
    
    model.add(ELU())              
    model.add(Convolution2D(64, 3, 3, 
                            subsample= (1,1), 
                            border_mode = 'valid',
                            init = 'he_normal',
                            name = 'conv5'))
              
              
    model.add(Flatten(name = 'flatten'))
    model.add(ELU())
    model.add(Dense(100, init = 'he_normal', name = 'fc1'))
    model.add(ELU())
    model.add(Dense(50, init = 'he_normal', name = 'fc2'))
    model.add(ELU())
    model.add(Dense(10, init = 'he_normal', name = 'fc3'))
    model.add(ELU())
    
    # do not put activation at the end because we want to exact output, not a class identifier
    model.add(Dense(1, name = 'output', init = 'he_normal'))
    
    adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(optimizer = adam, loss = 'mse')

    return model


In [ ]:
print('len: ', len(X_valid_data))
print('len train :', len(X_train_data))

In [ ]:
val_size = len(X_valid_data)
valid_generator = generate_validation_data(X_valid_data)

In [ ]:
model = nvidia_model()
for i in range(3):
    train_generator = generate_training_data(X_train_data, 256)
    history = model.fit_generator(
            train_generator, 
            samples_per_epoch = 20480, # try putting the whole thing in here in the future
            nb_epoch = 6,
            validation_data = valid_generator,
            nb_val_samples = val_size)
    print(history)
    
    model.save_weights('model-weightsU3.h5')
    model.save('modelU3.h5')

### Test out sequential running

In [ ]:
# run this one
model = nvidia_model()
for i in range(3):
    train_generator = generate_training_data(X_train_data, 256)
    history = model.fit_generator(
            train_generator, 
            samples_per_epoch = 20480, # try putting the whole thing in here in the future
            nb_epoch = 6,
            validation_data = valid_generator,
            nb_val_samples = val_size)
    print(history)


In [ ]:
train_recovery_generator = generate_training_data(X_train_data_recovery, 16)
history = model.fit_generator(train_recovery_generator, 
            samples_per_epoch = len(X_train_data_recovery), # try putting the whole thing in here in the future
            nb_epoch = 8,
            validation_data = valid_generator,
            nb_val_samples = val_size)

model.save_weights('model-weightsU5.h5')
model.save('modelU5.h5')

In [ ]:
model = nvidia_model()
for i in range(3):
    train_generator = generate_training_data(X_train_data_recovery, 256)
    history = model.fit_generator(
            train_generator, 
            samples_per_epoch = 20480, # try putting the whole thing in here in the future
            nb_epoch = 6,
            validation_data = valid_generator,
            nb_val_samples = val_size)
    print(history)
    
    model.save_weights('model-weightsU3.h5')
    model.save('modelU3.h5')

## Visualizing Loss

In [ ]:
print(history.history.keys())

### plot the training and validation loss for each epoch
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()

In [ ]:
print(model.summary())

In [ ]:
print('yo')